In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 4.3 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, LSTM, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras_tuner.tuners import BayesianOptimization
from keras.optimizer_v2 import adam as adam_v2
import os
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_org = pd.read_csv("/content/drive/MyDrive/DC2/all_variables.csv")
df_org

Mounted at /content/drive


,Year,Month,LSOA_code,LSOA_name,Type_I_crime_amount,Total_month,Type_II_crime_amount
0,2011,1,E01000001,City of London 001A,2,1,8
1,2011,2,E01000001,City of London 001A,1,2,8
2,2011,3,E01000001,City of London 001A,6,3,4
3,2011,4,E01000001,City of London 001A,1,4,4
4,2011,6,E01000001,City of London 001A,1,6,15
...,...,...,...,...,...,...,...
3286689,2021,6,W01001958,Swansea 025H,6,126,16
3286690,2021,7,W01001958,Swansea 025H,4,127,11
3286691,2021,8,W01001958,Swansea 025H,9,128,10
3286692,2021,9,W01001958,Swansea 025H,4,129,16


In [ ]:
# E01000005 for Urban
# E01030759 for Town and Fringe
# E01020795 for Village Hamlet & Isolated Dwellings

In [ ]:
# Extract the data from a given LSOA code
df_need = df_org[df_org['LSOA_code'] == 'E01020795'].reset_index(drop = True)
df_need = df_need[['Total_month', 'Type_I_crime_amount', 'Type_II_crime_amount']]
df_need = df_need[df_need['Total_month'] <= 108]
df_need.drop(columns = 'Total_month', inplace = True)
df_need

,Type_I_crime_amount,Type_II_crime_amount
0,3,18
1,4,22
2,5,15
3,3,28
4,2,30
...,...,...
103,18,24
104,18,11
105,19,31
106,30,20


In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
          break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i: end_ix], sequences[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Hyperparameter searching if necessary

def build_model(hp):
    # Build the LSTM model with 2 LSTM layers and 1 Dense layer
    input = Input(shape = (12, 2))
    l1 = LSTM(units = hp.Int('units_1', min_value = 50, max_value = 300, step = 50), return_sequences = True, dropout = 0.2)(input)
    b1 = BatchNormalization()(l1)
    l2 = LSTM(units = hp.Int('units_2', min_value = 50, max_value = 300, step = 50), return_sequences = False, dropout = 0.2)(b1)
    b2 = BatchNormalization()(l2)
    output = Dense(2)(b2)
    model = Model(input, output)

    # The optimizer is Adam
    opt = adam_v2.Adam(learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 2e-2, 2e-3, 2e-4]))
    model.compile(loss = 'mse', optimizer = opt, metrics = ['mse'])

    return model

def find_best_model():
    # Example of splitting the data when time step is 12 months: 
    # Training data has data from 1st month to 24th month
    # Val data has data from 13th month to 36th month
    # Test data predicts new data using data from 25th month to 36th month
    train_data = df_need.to_numpy()[: 96]
    val_data = df_need.to_numpy()[84: 108]
    # Split x and y arrays using sliding window approach
    x_train, y_train = split_sequences(train_data, 12)
    x_val, y_val = split_sequences(val_data, 12)
    bayesian_opt_tuner = BayesianOptimization(build_model, objective = 'mse',
        max_trials = 216,
        executions_per_trial = 1,
        directory = '/content/drive/MyDrive/DC2/tune/Rural/',
        project_name = 'kerastuner_bayesian_poc',
        overwrite = True)

    bayesian_opt_tuner.search(x_train, y_train, epochs = 500, validation_data = (x_val, y_val))

    bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models = 1)
    model = bayes_opt_model_best_model[0]
    return model

#model = find_best_model()
#model.summary()

In [ ]:
def train(df, train_nr, prediction_nr, comparison, model_exist, n_steps=12):
    with tf.device(device_name):
        # Example of splitting the data when time step is 12 months: 
        # Training data has data from 1st month to 24th month
        # Val data has data from 13th month to 36th month
        # Test data predicts new data using data from 25th month to 36th month
        train_data = df.to_numpy()[: train_nr - n_steps]
        val_data = df.to_numpy()[train_nr - n_steps * 2: train_nr]
        # Split x and y arrays using sliding window approach
        x_train, y_train = split_sequences(train_data, n_steps)
        x_val, y_val = split_sequences(val_data, n_steps)

        if model_exist == "Nil":
            # Build the LSTM model with 2 LSTM layers and 1 Dense layer
            input = Input(shape = (n_steps, 2))
            l1 = LSTM(50, return_sequences = True, dropout = 0.2)(input)
            b1 = BatchNormalization()(l1)
            l2 = LSTM(50, return_sequences = False, dropout = 0.2)(b1)
            b2 = BatchNormalization()(l2)
            output = Dense(2)(b2)
            model = Model(input, output)

            # The optimizer is Nadam
            opt = adam_v2.Adam(learning_rate = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)

            # The loss used in model training is mean_squared_error because it is crime number prediction
            model.compile(loss = 'mse', optimizer = opt)

        else:
            model = model_exist
        
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 200)
        checkpoint_filepath = '/content/drive/MyDrive/DC2/model/Rural/' + str(train_nr) + '/weights.{epoch:02d}.h5'
        model_checkpoint_callback = ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'val_loss', mode = 'min', save_best_only = True)
        lr_reducer = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 50, verbose = 0, mode = 'auto', min_delta = 0.0001, cooldown = 0, min_lr = 0)

        # Fit the model with 1000 epoches and batch size 64
        # Validation data is used here for evaluation during the training process
        model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 1000, batch_size = 32, callbacks = [model_checkpoint_callback, lr_reducer, early_stopping])

        # Look for all file directories in the folder
        directory = "/content/drive/MyDrive/DC2/model/Rural/" + str(train_nr) + '/'
        all_file_lst = []
        for root, subdirectories, files in os.walk(directory):
            for file in files:
                all_file_lst.append(os.path.join(root, file))
        # Load the best model trained
        model = load_model(all_file_lst[-1])

        # Make predictions from testing data
        # Pred x is a list of predicted data used
        # Pred y is a list of predicted result
        # Get the last 12 results from overall training data for prediction data in pred x
        pred_x = np.array(list(val_data[-(n_steps): ]))
        pred_x = pred_x.reshape((1, pred_x.shape[0], pred_x.shape[1]))
        # Initialize pred y as empty array
        pred_y = np.empty((1, 1, 2), int)
        for i in range(prediction_nr):
            # Always round off the latest prediction to integer because number of crime can only be integer
            prediction = np.round(model.predict(pred_x))
            # Update pred y 
            pred_y = np.append(pred_y, [prediction], axis = 1)
            # Update pred x by feeding the latest prediction result as latest feature
            pred_x = np.append(pred_x, [prediction], axis = 1)
            # Update pred x by removing the 1st row because of the timestep size
            pred_x = pred_x[: , 1: , : ]
        # Remove the 1st row of pred y because when initializing array, random numbers are assigned which are not actual results
        pred_y = pred_y[: , 1: , : ]
        
        df_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
        df_predict['Type_I_crime_amount'] = np.round(pred_y[: , : , 0])[0]
        df_predict['Type_II_crime_amount'] = np.round(pred_y[: , : , 1])[0]

        if comparison == False:
            return df_predict
        else:
            # Compute RMSE, MAE and R2 by comparing predicted result from actual crime number
            y_true = df[train_nr: (train_nr + prediction_nr)]
            rmse_1 = math.sqrt(mean_squared_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            rmse_2 = math.sqrt(mean_squared_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            mae_1 = mean_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            mae_2 = mean_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_1 = median_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_2 = median_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_1 = r2_score(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_2 = r2_score(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            return df_predict, rmse_1, rmse_2, mae_1, mae_2, medae_1, medae_2, r2_1, r2_2

In [ ]:
%%time
df_compare_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
rmse_lst_1 = []
rmse_lst_2 = []
mae_lst_1 = []
mae_lst_2 = []
medae_lst_1 = []
medae_lst_2 = []
r2_lst_1 = []
r2_lst_2 = []

# Month Forward-Chaining Cross Validation Approach is used below
for i in tqdm(range(6)):
    df_small, rmse_1_small, rmse_2_small, mae_1_small, mae_2_small, medae_1_small, medae_2_small, r2_1_small, r2_2_small = train(df_need, 36 + 12 * i, 12, True, "Nil")
    df_compare_predict = pd.concat([df_compare_predict, df_small])
    rmse_lst_1.append(rmse_1_small)
    rmse_lst_2.append(rmse_2_small)
    mae_lst_1.append(mae_1_small)
    mae_lst_2.append(mae_2_small)
    medae_lst_1.append(medae_1_small)
    medae_lst_2.append(medae_2_small)    
    r2_lst_1.append(r2_1_small)
    r2_lst_2.append(r2_2_small)

df_compare_predict

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
1/1 [==============================] - 13s 13s/step - loss: 81.2180 - val_loss: 174.3965 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 116ms/step - loss: 83.2250 - val_loss: 173.4499 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 56ms/step - loss: 82.0801 - val_loss: 174.0912 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 54ms/step - loss: 79.2453 - val_loss: 173.9387 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 49ms/step - loss: 77.6510 - val_loss: 173.9628 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 49ms/step - loss: 76.4827 - val_loss: 174.8307 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 51ms/step - loss: 75.1964 - val_loss: 175.5065 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 45ms/step - loss: 74.6671 - val_loss: 176.2879 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 46ms/step - loss: 76.105

 17%|█▋        | 1/6 [00:43<03:36, 43.21s/it]

Epoch 1/1000
1/1 [==============================] - 5s 5s/step - loss: 131.5135 - val_loss: 117.7014 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 45ms/step - loss: 134.8604 - val_loss: 120.3741 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 45ms/step - loss: 127.7598 - val_loss: 121.3878 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 50ms/step - loss: 127.8520 - val_loss: 120.6705 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 46ms/step - loss: 123.8262 - val_loss: 119.6421 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 49ms/step - loss: 120.0650 - val_loss: 118.6006 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 47ms/step - loss: 124.7327 - val_loss: 118.0536 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 129ms/step - loss: 123.5575 - val_loss: 117.1336 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 127ms/step - loss:

 33%|███▎      | 2/6 [01:32<03:06, 46.55s/it]

...

Epoch 1/1000
2/2 [==============================] - 5s 1s/step - loss: 156.4514 - val_loss: 480.2941 - lr: 0.0050
Epoch 2/1000
2/2 [==============================] - 0s 115ms/step - loss: 151.2407 - val_loss: 467.8654 - lr: 0.0050
Epoch 3/1000
2/2 [==============================] - 0s 119ms/step - loss: 148.5696 - val_loss: 458.1617 - lr: 0.0050
Epoch 4/1000
2/2 [==============================] - 0s 114ms/step - loss: 152.2946 - val_loss: 454.5522 - lr: 0.0050
Epoch 5/1000
2/2 [==============================] - 0s 40ms/step - loss: 150.1097 - val_loss: 456.2034 - lr: 0.0050
Epoch 6/1000
2/2 [==============================] - 0s 41ms/step - loss: 145.7617 - val_loss: 457.9873 - lr: 0.0050
Epoch 7/1000
2/2 [==============================] - 0s 43ms/step - loss: 144.5754 - val_loss: 457.5769 - lr: 0.0050
Epoch 8/1000
2/2 [==============================] - 0s 44ms/step - loss: 140.9381 - val_loss: 456.6108 - lr: 0.0050
Epoch 9/1000
2/2 [==============================] - 0s 47ms/step - loss

 83%|████████▎ | 5/6 [03:36<00:42, 42.11s/it]

Epoch 1/1000
3/3 [==============================] - 5s 635ms/step - loss: 209.0695 - val_loss: 251.5187 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 63ms/step - loss: 199.0887 - val_loss: 250.7147 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 70ms/step - loss: 195.3803 - val_loss: 246.5474 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 77ms/step - loss: 188.1829 - val_loss: 240.9647 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 68ms/step - loss: 184.2573 - val_loss: 234.6656 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 26ms/step - loss: 181.5599 - val_loss: 239.2754 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 26ms/step - loss: 175.1440 - val_loss: 245.4683 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 28ms/step - loss: 175.5948 - val_loss: 236.4807 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 64ms/step - loss

100%|██████████| 6/6 [04:09<00:00, 41.54s/it]

CPU times: user 3min 3s, sys: 14.7 s, total: 3min 17s
Wall time: 4min 9s


In [ ]:
rmse_lst_1, rmse_lst_2

([3.068658773253661,
  2.041241452319315,
  5.7081812631812365,
  13.89544289806314,
  3.582364210034113,
  10.115993936995679],
 [9.742518497116988,
  11.782755761422425,
  12.747548783981962,
  19.01096175017631,
  7.75134396948899,
  8.741662694628904])

In [ ]:
mae_lst_1, mae_lst_2

([2.4166666666666665,
  1.8333333333333333,
  5.083333333333333,
  11.083333333333334,
  3.1666666666666665,
  8.666666666666666],
 [7.75,
  10.666666666666666,
  11.166666666666666,
  18.083333333333332,
  6.25,
  6.583333333333333])

In [ ]:
medae_lst_1, medae_lst_2

([2.0, 2.0, 4.0, 8.5, 3.5, 7.5], [5.5, 10.5, 9.5, 18.0, 5.5, 4.5])

In [ ]:
r2_lst_1, r2_lst_2

([-0.6396614268440148,
  -0.28205128205128216,
  -0.7592800899887515,
  -2.0855620907779384,
  -0.203125,
  -1.5219921273318504],
 [-1.723251643753736,
  -3.4112974404236542,
  -3.298310066127848,
  -9.606072957000203,
  -2.4957575757575756,
  -0.44865718799368093])

In [ ]:
rmse_1 = sum(rmse_lst_1) / len(rmse_lst_1)
rmse_2 = sum(rmse_lst_2) / len(rmse_lst_2)
mae_1 = sum(mae_lst_1) / len(mae_lst_1)
mae_2 = sum(mae_lst_2) / len(mae_lst_2)
medae_1 = sum(medae_lst_1) / len(medae_lst_1)
medae_2 = sum(medae_lst_2) / len(medae_lst_2)
r2_1 = sum(r2_lst_1) / len(r2_lst_1)
r2_2 = sum(r2_lst_2) / len(r2_lst_2)

print("Type I crime amount: \n RMSE: " + str(rmse_1) + "\n MAE: " + str(mae_1) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_1) + 
      "\n \n Type II crime amount: \n RMSE: " + str(rmse_2) + "\n MAE: " + str(mae_2) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_2))

Type I crime amount: 
 RMSE: 6.401980422307858
 MAE: 5.375
 MEDAE: 4.583333333333333
 R2: -0.9152786694989729
 
 Type II crime amount: 
 RMSE: 11.629465242802597
 MAE: 10.083333333333332
 MEDAE: 4.583333333333333
 R2: -3.49722447850945


In [ ]:
df_real_predict = train(df_need, 108, 24, False, "Nil")
df_real_predict

Epoch 1/1000
3/3 [==============================] - 5s 613ms/step - loss: 212.8300 - val_loss: 313.6849 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 25ms/step - loss: 205.4382 - val_loss: 316.2347 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 25ms/step - loss: 199.9999 - val_loss: 315.7409 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 29ms/step - loss: 195.6047 - val_loss: 314.0114 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 25ms/step - loss: 192.3415 - val_loss: 315.7806 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 68ms/step - loss: 188.3552 - val_loss: 307.6063 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 64ms/step - loss: 184.6285 - val_loss: 304.3809 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 68ms/step - loss: 180.1067 - val_loss: 286.0780 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 61ms/step - loss

,Type_I_crime_amount,Type_II_crime_amount
0,12.0,17.0
1,12.0,17.0
2,12.0,17.0
3,12.0,17.0
4,12.0,17.0
5,12.0,17.0
6,12.0,17.0
7,12.0,17.0
8,12.0,17.0
9,12.0,17.0


In [ ]:
df_compare_predict.to_csv("/content/drive/MyDrive/DC2/rural_old_prediction.csv", index = False)
df_real_predict.to_csv("/content/drive/MyDrive/DC2/rural_new_prediction.csv", index = False)